In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import numpy as np
from collections import Counter
from tqdm import tqdm
import torchvision.models as models
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import convnext_tiny
from sklearn.model_selection import train_test_split
import timm

In [2]:
SEED = 42 
np.random.seed(SEED) 
torch.manual_seed(SEED)

In [3]:
class DERMNET_DATASET(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        super().__init__()
        self.root_dir = root_dir
        self.train = train
        self.transform = transform

        self.labels_dict = {
            'Light Diseases and Disorders of Pigmentation': 0, 
            'Lupus and other Connective Tissue diseases': 1,
            'Acne and Rosacea Photos': 2,
            'Systemic Disease': 3,
            'Poison Ivy Photos and other Contact Dermatitis': 4,
            'Vascular Tumors': 5,
            'Urticaria Hives': 6,
            'Atopic Dermatitis Photos': 7,
            'Bullous Disease Photos': 8,
            'Hair Loss Photos Alopecia and other Hair Diseases': 9,
            'Tinea Ringworm Candidiasis and other Fungal Infections': 10,
            'Psoriasis pictures Lichen Planus and related diseases': 11,
            'Melanoma Skin Cancer Nevi and Moles': 12,
            'Nail Fungus and other Nail Disease': 13,
            'Scabies Lyme Disease and other Infestations and Bites': 14,
            'Eczema Photos': 15,
            'Exanthems and Drug Eruptions': 16,
            'Herpes HPV and other STDs Photos': 17,
            'Seborrheic Keratoses and other Benign Tumors': 18,
            'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 19,
            'Vasculitis Photos': 20,
            'Cellulitis Impetigo and other Bacterial Infections': 21,
            'Warts Molluscum and other Viral Infections': 22
        }

        dataset_type = 'train' if self.train else 'test'
        dataset_dir = os.path.join(root_dir, dataset_type)

        # Tạo một dictionary để lưu danh sách ảnh cho từng nhãn
        label_to_images = {label: [] for label in range(len(self.labels_dict))}

        # Duyệt qua các thư mục con và lưu đường dẫn ảnh theo nhãn tương ứng
        for class_name, class_idx in self.labels_dict.items():
            class_folder = os.path.join(dataset_dir, class_name)
            if os.path.isdir(class_folder):
                image_paths = sorted([os.path.join(class_folder, img) 
                                      for img in os.listdir(class_folder) 
                                      if img.endswith(('.png', '.jpg', '.jpeg'))])
                label_to_images[class_idx].extend(image_paths)

        # Tạo danh sách ảnh và nhãn xen kẽ
        self.image_paths = []
        self.labels = []
        for label, images in label_to_images.items():
            for img in images:
                self.image_paths.append(img)
                self.labels.append(label)

    def __len__(self):
        return len(self.labels)  # Tổng số lượng ảnh

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Mở ảnh và áp dụng transform nếu có
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label)

        return image, label

    def show_image(self, idx):
        image, label = self.__getitem__(idx)
        keys = [k for k, v in self.labels_dict.items() if v == label]
        
        # Hiển thị ảnh
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Label: {keys[0]}")
        plt.show()


In [4]:
from torch.utils.data import Dataset, Subset

class CustomSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = Subset(dataset, indices)
        self.indices = indices
        self.labels_dict = dataset.labels_dict  # Giữ nguyên labels_dict từ dataset gốc
        self.dataset_cls = dataset  # Duy trì tham chiếu tới dataset gốc

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        return self.dataset[idx]

    def get_class_counts(self):
        """Tính toán số lượng ảnh trong mỗi lớp."""
        class_counts = {class_name: 0 for class_name in self.labels_dict.keys()}
        for idx in self.indices:
            label = self.dataset_cls.labels[idx]
            class_name = [k for k, v in self.labels_dict.items() if v == label][0]
            class_counts[class_name] += 1
        return class_counts

    def show_image(self, idx):
        """Hiển thị ảnh."""
        actual_idx = self.indices[idx]
        self.dataset_cls.show_image(actual_idx)

In [5]:
import torchvision.transforms as transforms 
# Định nghĩa các bước tiền xử lý 
transform = transforms.Compose([ transforms.Resize((128, 128)),   
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [6]:
test_dataset = DERMNET_DATASET('/kaggle/input/dermnet', train=False, transform=transform)

In [7]:
batch_size = 128

test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

print(f'Test DataLoader: {len(test_dataloader.dataset)} images')


Test DataLoader: 4002 images


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
from torchvision.models import convnext_tiny, densenet121, mobilenet_v2, resnet50
import timm

In [10]:
class ConvNeXtTinyModel(nn.Module):
    def __init__(self, num_classes=23, pretrained=True, freeze_backbone=False):
        super(ConvNeXtTinyModel, self).__init__()
        
        # Load pre-trained ConvNeXt Tiny model
        self.model = convnext_tiny(pretrained=pretrained)
        
        # Optionally freeze backbone layers
        if freeze_backbone:
            for param in self.model.features.parameters():
                param.requires_grad = False
        
        # Replace the classifier layer (fully connected layer) to match the number of classes
        in_features = self.model.classifier[2].in_features
        self.model.classifier[2] = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        return self.model(x)


class DenseNet121(nn.Module):
    def __init__(self, num_classes=10, pretrained=True, frozen=False):
        super(DenseNet121, self).__init__()
        self.model = densenet121(pretrained=pretrained)
        
        if frozen:
            for param in self.model.features.parameters():  # Freeze chỉ các lớp feature extractor
                param.requires_grad = False
        
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        # Truyền input qua mô hình DenseNet121
        return self.model(x)

class MobileNetV2(nn.Module):
    def __init__(self, num_classes=10, frozen=False):
        super(MobileNetV2, self).__init__()
        self.model = mobilenet_v2(pretrained=True)
        if frozen:
            for param in self.model.parameters():
                param.requires_grad = False
        num_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

class EfficientNetV2(nn.Module):
    def __init__(self, num_classes=10, frozen=False):
        super(EfficientNetV2, self).__init__()
        self.model = timm.create_model('efficientnetv2_rw_t', pretrained=True)
        if frozen:
            for param in self.model.parameters():
                param.requires_grad = False
            for param in self.model.classifier.parameters():
                param.requires_grad = True
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

class ResNet50(nn.Module):
    def __init__(self, num_classes=10, frozen=False):
        super(ResNet50, self).__init__()
        self.model = resnet50(pretrained=True)
        if frozen:
            for param in self.model.parameters():
                param.requires_grad = False
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

class ViTClassifier128(nn.Module):
    def __init__(self, num_classes=23, pretrained=True, img_size=128, freeze_backbone=False):
        super(ViTClassifier128, self).__init__()
        
        # Tải mô hình ViT với kích thước ảnh 128x128
        self.vit = timm.create_model(
            'vit_base_patch16_224',
            pretrained=pretrained,
            img_size=img_size
        )
        
        # Freeze backbone nếu cần
        if freeze_backbone:
            for param in self.vit.parameters():
                param.requires_grad = False

        # Lấy số lượng đầu ra từ lớp head
        in_features = self.vit.head.in_features
        
        # Thay thế lớp head bằng lớp phân loại tùy chỉnh
        self.vit.head = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.vit(x)

In [11]:
# Load toàn bộ mô hình
convnext_frozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_convnext_frozen.path').to(device)
convnext_unfrozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_convnext_unfrozen.path').to(device)
densenet_frozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_model_densenet_frozen.pth').to(device)
densenet_unfrozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_model_densenet_unfrozen.pth').to(device)
mobilenet_frozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_model_mobile.pth').to(device)
mobilenet_unfrozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_model_unfrozen_mobile.pth').to(device)
vit_frozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_swin_frozen.path').to(device)
efficientnet_frozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_model_efficient_frozen.path').to(device)
efficientnet_unfrozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_model_efficient_unfrozen.path').to(device)
resnet50_frozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/resnet50frozen.pth').to(device)
resnet50_unfrozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/resnet50unfrozen.pth').to(device)

/tmp/ipykernel_23/203563616.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  convnext_frozen = torch.load('/kaggle/input/basemodels/other/default/1/best_model/best_convne

In [12]:
def precompute_model_outputs(base_models, loader, device):
    """
    Lưu nhãn dự đoán của từng mô hình cơ sở.
    """
    model_outputs = {}
    labels_list = []

    for images, labels in tqdm(loader, desc="Extracting Outputs"):
        images = images.to(device)

        for i, model in enumerate(base_models):
            model.eval()
            with torch.no_grad():
                # Dự đoán và lấy nhãn dự đoán (argmax trên logits)
                logits = model(images).cpu().numpy()
                predicted_labels = np.argmax(logits, axis=1)  # Lấy nhãn
                if i not in model_outputs:
                    model_outputs[i] = []
                model_outputs[i].append(predicted_labels)
        
        labels_list.append(labels.cpu().numpy())

    # Gộp toàn bộ batch lại
    for i in model_outputs:
        model_outputs[i] = np.hstack(model_outputs[i])  # Chuyển thành 1 mảng 1D
    labels = np.hstack(labels_list)  # Nhãn thực tế

    return model_outputs, labels



# Danh sách các mô hình
base_models = [
    convnext_frozen, convnext_unfrozen, densenet_frozen, densenet_unfrozen,
    mobilenet_frozen, mobilenet_unfrozen, efficientnet_frozen,
    efficientnet_unfrozen, resnet50_frozen, resnet50_unfrozen, vit_frozen
]
base_models_string = [
    'convnext_frozen', 'convnext_unfrozen', 'densenet_frozen', 'densenet_unfrozen',
    'mobilenet_frozen', 'mobilenet_unfrozen', 'efficientnet_frozen',
    'efficientnet_unfrozen', 'resnet50_frozen', 'resnet50_unfrozen', 'vit_frozen'
]

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def fitness_function(chromosome, train_outputs_dict, train_labels, val_outputs_dict, val_labels, cache):
    """
    Tính fitness cho một tổ hợp mô hình (chromosome) chỉ dựa trên nhãn.
    """
    chromosome_key = tuple(chromosome.tolist())
    if chromosome_key in cache:
        return cache[chromosome_key]
    
    if sum(chromosome) == 0:  # Không có mô hình nào được chọn
        return 0

    # Lấy các nhãn từ train_labels và val_labels dựa trên chromosome
    selected_train_labels = np.array([train_outputs_dict[i] for i in range(len(chromosome)) if chromosome[i] == 1])
    selected_val_labels = np.array([val_outputs_dict[i] for i in range(len(chromosome)) if chromosome[i] == 1])
    if selected_train_labels.shape[0] == 0:  # Không có nhãn nào được chọn
        cache[chromosome_key] = 0
        return 0

    # Huấn luyện Logistic Regression chỉ với các nhãn
    clf = LogisticRegression(max_iter=10000, solver='liblinear', random_state=42)
    clf.fit(selected_train_labels.T, train_labels)  # Huấn luyện với nhãn

    # Dự đoán trên tập validation
    val_predictions = clf.predict(selected_val_labels.T)

    # Tính độ chính xác
    val_accuracy = accuracy_score(val_labels, val_predictions)

    cache[chromosome_key] = val_accuracy
    return val_accuracy

    
def genetic_algorithm(base_models, base_models_string,train_outputs_dict, train_labels,val_outputs_dict, val_labels, device, num_generations=10, population_size=20):
    """
    Genetic Algorithm để tìm tổ hợp mô hình tốt nhất.
    """
    # Lưu trước đầu ra của mô hình


    num_models = len(base_models)
    def initialize_population_fixed_selection(population_size, num_models, k):
        population = np.zeros((population_size, num_models), dtype=int)
        for i in range(population_size):
            selected_indices = np.random.choice(num_models, k, replace=False)
            population[i, selected_indices] = 1
        return population

    # Sử dụng
    population = initialize_population_fixed_selection(population_size, num_models, k=4)

    best_chromosome = None
    best_fitness = 0
    cache = {}

    for generation in tqdm(range(num_generations), desc="Generations"):
        fitness_scores = []

        for chromosome in tqdm(population, desc=f"Evaluating Population (Generation {generation})", leave=False):
            fitness = fitness_function(
                chromosome, train_outputs_dict, train_labels, val_outputs_dict, val_labels, cache
            )
            fitness_scores.append(fitness)

        gen_best_fitness = max(fitness_scores)
        gen_best_chromosome = population[np.argmax(fitness_scores)]

        if gen_best_fitness > best_fitness:
            best_fitness = gen_best_fitness
            best_chromosome = gen_best_chromosome

        selected_models = [base_models_string[i] for i in range(num_models) if gen_best_chromosome[i] == 1]
        print(f"Generation {generation}, Best fitness: {gen_best_fitness:.4f}, Models: {selected_models}")

        # Chọn bố mẹ và tạo thế hệ mới
        parents = population[np.argsort(fitness_scores)[-10:], :]
        next_generation = []
        for _ in range(population_size // 2):
            parent1, parent2 = random.choices(parents, k=2)
            crossover_point = random.randint(1, num_models - 1)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

            mutation_rate = 0.1
            if random.random() < mutation_rate:
                child1[random.randint(0, num_models - 1)] ^= 1
            if random.random() < mutation_rate:
                child2[random.randint(0, num_models - 1)] ^= 1

            next_generation.append(child1)
            next_generation.append(child2)

        population = np.array(next_generation)

    print(f"Best chromosome: {best_chromosome}")
    print(f"Best fitness: {best_fitness:.4f}")
    return best_chromosome, best_fitness, len(cache)


In [14]:
test_outputs_dict, test_labels = precompute_model_outputs(base_models, test_dataloader, device)

Extracting Outputs: 100%|██████████| 32/32 [01:16<00:00,  2.38s/it]


In [15]:
import json
with open("/kaggle/input/output/model_outputs.json", "r") as f:
    model_outputs = json.load(f)

# Chuyển list thành numpy arrays (nếu cần)
model_outputs = {
    model_name: {
        "predictions": np.array(data["predictions"]),
        "labels": np.array(data["labels"]),
    }
    for model_name, data in model_outputs.items()
}
train_outputs_dict = {}
train_labels = model_outputs['convnext_unfrozen']['labels']  # Nhãn thực tế

for i, model in enumerate(base_models_string):
    # Lấy nhãn dự đoán (argmax trên trục 1 của vector xác suất)
    train_outputs_dict[i] = np.argmax(model_outputs[model]['predictions'], axis=1)


In [54]:
import random
best_chromosome, best_fitness, length = genetic_algorithm(
    base_models_string=base_models_string,
    base_models=base_models,
    device=device,
    num_generations=100,
    population_size=20,
    train_outputs_dict = train_outputs_dict , 
    train_labels = train_labels,
    val_outputs_dict = test_outputs_dict, 
    val_labels = test_labels
)

Generations:   1%|          | 1/100 [00:08<14:04,  8.53s/it]                         

Generation 0, Best fitness: 0.2156, Models: ['convnext_unfrozen', 'densenet_unfrozen', 'efficientnet_frozen', 'efficientnet_unfrozen']



Generations:   2%|▏         | 2/100 [00:13<10:58,  6.72s/it]                         

Generation 1, Best fitness: 0.2164, Models: ['densenet_frozen', 'efficientnet_frozen', 'efficientnet_unfrozen']



Generations:   3%|▎         | 3/100 [00:19<09:41,  5.99s/it]                         

...

Generation 98, Best fitness: 0.2296, Models: ['efficientnet_unfrozen', 'resnet50_frozen']



Evaluating Population (Generation 99):   0%|          | 0/20 [00:00<?, ?it/s]
                                                                             

Generation 99, Best fitness: 0.2296, Models: ['efficientnet_unfrozen', 'resnet50_frozen']


Generations: 100%|██████████| 100/100 [00:30<00:00,  3.25it/s]

Best chromosome: [0 0 0 0 0 0 0 1 1 0 0]
Best fitness: 0.2296
